In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer
from xgboost import XGBRegressor
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import ElasticNet
from sklearn import linear_model
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# read data

In [ ]:
train=pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/data.csv")
submission=pd.read_csv("/kaggle/input/tabular-playground-series-jun-2022/sample_submission.csv")

# check data

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
train.dtypes

In [ ]:
train.dtypes

In [ ]:
s = (train.dtypes != 'float64')
object_cols = list(s[s].index)

print("train Categorical variables:")
print(object_cols)

In [ ]:
submission.head()

# EDA

**how many columns have null?**

In [ ]:
cols_with_missing = [col for col in train.columns if train[col].isnull().any()]
print('number of column which have null is : ',len(cols_with_missing))

*columns that have null cell is 55  
26 column have no null cell  
(but 1 of 26 columns is row_id)*

**how many null cell each column have?**

In [ ]:
count_list=[]
for col in cols_with_missing:
    count=train[col].isnull().tolist().count(True)
    count_list.append(count)
df=pd.DataFrame(columns=['missing_col', 'count'])
df['missing_col']=cols_with_missing
df['count']=count_list
df.head()

In [ ]:
df.describe()

In [ ]:
sns.histplot(data=df['count'])

*55 columns have null cell almost same number*

**how many rows have null cell?**

In [ ]:
row_with_missing = [row for row in range(train.shape[0]) if train.loc[row,:].isnull().any()]
print('number of row which have null is : ',len(row_with_missing))

In [ ]:
count_list=[]
for row in row_with_missing:
    count=train.loc[row,:].isnull().tolist().count(True)
    count_list.append(count)
df=pd.DataFrame(count_list)
df['count_col']=1

In [ ]:
print(df.groupby(0).sum().T)
sns.histplot(data=df[0])

*about 64% rows have null  
almost rows at that have null,  have 1 or 2 cells that are null  
and some of them have 9 cells that are null*

**which column have no null?**

In [ ]:
full_col_df=train.drop(cols_with_missing,axis=1)
full_col_df=full_col_df.drop('row_id',axis=1)
full_col_df.head()

*column F_2_0 ～ F_2_24 have no null value*

In [ ]:
full_col_df.describe().T

**check heatmap**

In [ ]:
sns.set(rc = {'figure.figsize':(15,8)})
df_corr=train.corr()
sns.heatmap(df_corr, vmax=1, vmin=-1, center=0)

*column F_2_0 ～ F_2_24 looks have correlation  
and F_4_0 ～ F_4_14 also have. it is interesting*

**check F_2_0 ～ F_2_24 column data histplot** 

In [ ]:
col_list=full_col_df.columns
ncols = 3
nrows = 8

fig, axes = plt.subplots(nrows, ncols, figsize=(40,50), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = col_list[r*ncols+c+1]
        sns.histplot(x=full_col_df[col], ax=axes[r, c])
        axes[r, c].set_ylabel('')
        axes[r, c].set_xlabel(col, fontsize=15, fontweight='bold')
        axes[r, c].tick_params(labelsize=10, width=0.5)
        axes[r, c].xaxis.offsetText.set_fontsize(20)
        axes[r, c].yaxis.offsetText.set_fontsize(20)
plt.show()

**check other than F_2_0 ～ F_2_24 column data histplot** 

In [ ]:
col_list=train[cols_with_missing].columns
ncols = 5
nrows = 11

fig, axes = plt.subplots(nrows, ncols, figsize=(30,45), facecolor='#EAEAF2')

for r in range(nrows):
    for c in range(ncols):
        col = col_list[r*ncols+c]
        sns.histplot(x=train[col], ax=axes[r, c])
        axes[r, c].set_ylabel('')
        axes[r, c].set_xlabel(col, fontsize=15, fontweight='bold')
        axes[r, c].tick_params(labelsize=10, width=0.5)
        axes[r, c].xaxis.offsetText.set_fontsize(20)
        axes[r, c].yaxis.offsetText.set_fontsize(20)
plt.show()

# prepare train data with Imputer

**I'll use SimpleImputer and IterativeImputer from scikit-learn**   
https://scikit-learn.org/stable/modules/classes.html#module-sklearn.impute

In [ ]:
simple_imputer = SimpleImputer(missing_values=np.nan, fill_value=None,strategy='median')
simple_inputed=pd.DataFrame(simple_imputer.fit_transform(train.iloc[:,1:]))
simple_inputed.columns=train.iloc[:,1:].columns

In [ ]:
# estimator = XGBRegressor(n_estimators=1000, 
#                          learning_rate=0.2, 
#                          max_depth=3, 
#                          random_state=42, 
#                          tree_method='gpu_hist')
# iterative_imputer = IterativeImputer(estimator=estimator, missing_values=np.nan, max_iter=10, random_state=42)

# iterative_inputed=pd.DataFrame(iterative_imputer.fit_transform(train.iloc[:,1:]))
# iterative_inputed.columns=train.iloc[:,1:].columns
# iterative_inputed.to_csv('iterative_inputed.csv')

iterative_inputed=pd.read_csv("../input/tps-jun22-eda-predict/iterative_inputed.csv")
iterative_inputed.drop('Unnamed: 0',axis=1,inplace=True)
iterative_inputed['null_count']=train.isnull().sum(axis=1)

In [ ]:
#output for next ver
iterative_inputed.to_csv('iterative_inputed.csv')

# train model

In [ ]:
df_score=pd.DataFrame()

for i in range(55):
    
    df_train = train.dropna(subset=[cols_with_missing[i]])
    df_iterative=iterative_inputed.loc[df_train.index,:]
    
    X=df_iterative.drop(cols_with_missing[i],axis=1)
    y=df_iterative[cols_with_missing[i]]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y,random_state=0,test_size=0.2)
    
    score_list=[]

    #use LGBMRegressor
    #from this my notebook ver29,I found LGBMRegressor perform really good for columns F_4_1~F_4_14
    #So I'll skip other columns fitting because it takes long time to fit
    if i >39:
        X=X.iloc[:,65:-1]
        X_train, X_valid, y_train, y_valid = train_test_split(X, y,random_state=0,test_size=0.2)
        rg_model = lgb.LGBMRegressor(num_leaves=200,
                                 subsample_freq=3,
                                 n_estimators=2000,
                                 metric='root_mean_squared_error')
        rg_model.fit(X_train, y_train)
        y_pred=rg_model.predict(X_valid)
        #print('LGBMRegressor',np.sqrt(mean_squared_error(y_valid,y_pred)))
        score_list.append(np.sqrt(mean_squared_error(y_valid,y_pred)))
        
    else:
        score_list.append(3)
        
    
    #use OrthogonalMatchingPursuit
    rg_model = OrthogonalMatchingPursuit(normalize=False)
    rg_model.fit(X_train, y_train)
    y_pred=rg_model.predict(X_valid)
    #print('OrthogonalMatchingPursuit',np.sqrt(mean_squared_error(y_valid,y_pred)))
    score_list.append(np.sqrt(mean_squared_error(y_valid,y_pred)))
    
    #use ElasticNet
    rg_model = ElasticNet()
    rg_model.fit(X_train, y_train)
    y_pred=rg_model.predict(X_valid)
    #print('ElasticNet',np.sqrt(mean_squared_error(y_valid,y_pred)))
    score_list.append(np.sqrt(mean_squared_error(y_valid,y_pred)))

    #use BayesianRidge
    rg_model = linear_model.BayesianRidge()
    rg_model.fit(X_train, y_train)
    y_pred=rg_model.predict(X_valid)
    #print('BayesianRidge',np.sqrt(mean_squared_error(y_valid,y_pred)))
    score_list.append(np.sqrt(mean_squared_error(y_valid,y_pred)))    
    
    df_score[cols_with_missing[i]]=score_list

df_score.loc['std']=train[cols_with_missing].std()
df_score

# compare score and search best one

**check best model at each columns**

In [ ]:
a=pd.concat([df_score.idxmin(),df_score.min()],axis=1)
a['diff_std']=np.abs(a[1]-df_score.loc['std'].T)
a

**it looks almost same score, OrthogonalMatchingPursuit & BayesianRidge & ElasticNet & just std**

In [ ]:
row_list=a[0].tolist()
diff_list=a['diff_std'].tolist()

# train model with all data

In [ ]:
df_score_submit=train.copy()

for i in range(55):
    
    y=train.dropna(subset=[cols_with_missing[i]])
    y=y[cols_with_missing[i]]
    X_train=iterative_inputed.loc[y.index,:]
    X_train=X_train.drop(cols_with_missing[i],axis=1)
    X_valid=iterative_inputed.drop(cols_with_missing[i],axis=1)
    
    if row_list[i]==0:
        rg_model = lgb.LGBMRegressor(num_leaves=300,
                                     subsample_freq=10,
                                     n_estimators=3000,
                                     metric='root_mean_squared_error')
        X_train=X_train.iloc[:,65:-1]
        X_valid=X_valid.iloc[:,65:-1]
        rg_model.fit(X_train, y)
        df_score_submit[cols_with_missing[i]]=rg_model.predict(X_valid)
    else:
        rg_model = OrthogonalMatchingPursuit(normalize=False)
        rg_model_E = ElasticNet(alpha=0.3, l1_ratio = 0.8, max_iter=300)
        rg_model_B = linear_model.BayesianRidge(tol = 0.0003,n_iter=300)
        
        rg_model.fit(X_train, y)
        rg_model_E.fit(X_train, y)
        rg_model_B.fit(X_train, y)
        
        if diff_list[i]<0.0001:
            df_score_submit[cols_with_missing[i]]=(rg_model.predict(X_valid)
                                                   +rg_model_E.predict(X_valid)
                                                   +rg_model_B.predict(X_valid)
                                                   +simple_inputed[cols_with_missing[i]])/4
        else:
            df_score_submit[cols_with_missing[i]]=(rg_model.predict(X_valid)
                                                   +rg_model_E.predict(X_valid)
                                                   +rg_model_B.predict(X_valid))/3

# make prediction for submit

In [ ]:
submission[['row','col']]=submission['row-col'].str.split('-', expand=True)
submission_list=[]
for i in range(1000000):
    submission_list.append(df_score_submit.loc[int(submission.loc[i,'row']),submission.loc[i,'col']])
submission['value']=submission_list
submission=submission.drop(['row','col'],axis=1)
submission.to_csv('submission.csv', index=False)